In [1]:
import numpy as np
import math
from fractions import Fraction
from tqdm import tqdm
import multiprocessing as mp
from haversine import haversine, Unit
import matplotlib.pyplot as plt
import os
import random
import matplotlib
from scipy.sparse import lil_matrix, csr_matrix
import gc
import time
from multiprocessing import shared_memory

In [2]:
# Astronomy-related constants

TE = 24 * 3600  # One Earth sidereal day in seconds

RE = 6371e3  # Mean radius of Earth in meters

u = 3.986e14  # Standard gravitational parameter (μ = GM) in m^3/s^2

K = RE / pow(u, 1/3) * pow(2 * np.pi, 2/3)  

eps = 25 * np.pi / 180  # Maximum elevation angle for ground access, in radians (25°)

In [3]:
# === Orbital mechanics ===

def satellite_period(h):
    # Satellite orbital period (h in meters)
    a = RE + h
    return float(2 * np.pi * (a**3 / u)**0.5)

def coverage_eta(T):
    # Coverage half angle (rad)
    return math.acos(K * math.cos(eps) / T**(2/3)) - eps

def approximate_ratio(a, b, precision=1e-3):
    # Approximate integer ratio of a/b
    if b == 0:
        raise ValueError("Denominator cannot be zero")
    ratio = Fraction(a, b).limit_denominator(int(1 / precision))
    return ratio.numerator, ratio.denominator

def alpha_gamma_to_lambda_phi(alpha, gamma, inc):
    # Convert orbit plane coords to lat/lon
    phi = math.asin(math.sin(inc) * math.sin(gamma))
    temp = math.atan2(math.cos(inc) * math.sin(gamma), math.cos(gamma))
    lamb = (temp + alpha) % (2 * math.pi)
    if lamb > math.pi:
        lamb -= 2 * math.pi
    return lamb, phi


# === Ground coverage logic ===

def is_cover(l_sat, p_sat, l_cell, p_cell, eta):
    # Determine whether (l_sat, p_sat) covers (l_cell, p_cell)
    if (p_sat - 2*eta) < -np.pi/2 or (p_sat + 2*eta) > np.pi/2 or \
       (l_sat - 2*eta) < -np.pi or (l_sat + 2*eta) > np.pi:
        d = haversine((np.degrees(p_sat), np.degrees(l_sat)),
                      (np.degrees(p_cell), np.degrees(l_cell)),
                      unit=Unit.METERS)
        return d <= eta * RE
    else:
        if (p_sat - 2*eta) <= p_cell <= (p_sat + 2*eta) and \
           (l_sat - 2*eta) <= l_cell <= (l_sat + 2*eta):
            d = haversine((np.degrees(p_sat), np.degrees(l_sat)),
                          (np.degrees(p_cell), np.degrees(l_cell)),
                          unit=Unit.METERS)
            return d <= eta * RE
    return False

def get_grid_id(lat_deg: float, lon_deg: float) -> int:
    # Map (lat, lon) to grid ID (0–120), row-major from NW to SE
    lon_deg = ((lon_deg + 180) % 360) - 180
    lat = 90
    row, col = -1, -1

    for i in range(11):
        step = 16 if i < 10 else 20
        if lat_deg <= lat and lat_deg > (lat - step) if i < 10 else lat_deg >= -90:
            row = i
            break
        lat -= step

    lon = -180
    for j in range(11):
        step = 32 if j < 10 else 40
        if lon_deg >= lon and lon_deg < (lon + step) if j < 10 else lon_deg <= 180:
            col = j
            break
        lon += step

    if row == -1 and lat_deg == -90: row = 10
    if col == -1 and lon_deg == 180: col = 10

    if row != -1 and col != -1:
        return row * 11 + col
    else:
        print(f"Warning: Could not determine grid for lat={lat_deg}, lon={lon_deg}")
        return -1

def create_new_grid_satellites(satellite_location):
    # Convert sat (lon, lat) → grid ID
    lon, lat = satellite_location
    grid_id = get_grid_id(np.degrees(lat), np.degrees(lon))
    if 0 <= grid_id < 121:
        return grid_id
    else:
        raise ValueError("Invalid satellite location")


# === Satellite ground supply generation ===

def cal_supply_all_cell(h, beta, a0):
    '''
    Compute stable subsatellite point coverage over one orbital cycle
    Returns a list of [covered_cell_list, lon, lat] for each slot
    '''
    T = satellite_period(h * 1e3)
    p, q = approximate_ratio(int(T), TE)
    eta = coverage_eta(T)
    step = 2 * eta
    inc = beta
    total_supply = []

    num_slots = int(2 * np.pi * q / step)

    for i in range(num_slots):
        t = step * i / (2 * np.pi / T)
        a_sat = (a0 - (2 * np.pi / TE) * t) % (2 * np.pi)
        b_sat = (step * i) % (2 * np.pi)
        l_sat, p_sat = alpha_gamma_to_lambda_phi(a_sat, b_sat, inc)
        cid = create_new_grid_satellites([l_sat, p_sat])
        total_supply.append([[ [cid, 1.0] ], l_sat, p_sat])

    return np.asarray(total_supply, dtype="object")


# === Multiprocessing coverage computation ===

def recover_fulltime(data):
    # Aggregate full-time user coverage over time
    random_numbers, cover, n_length = data
    selected_cover = lil_matrix((1, n_length), dtype=np.float64)
    sat_location = []

    for t in range(time_split):
        for num in random_numbers:
            num = (num + t) % time_split
            sat_location.append([cover[num][1], cover[num][2]])
            for idx, users in cover[num][0]:
                selected_cover[0, t * len(demand_list) + idx] += users

    return [selected_cover.tocsr(), sat_location]

def generate_baseset(args):
    # Build base unit from file and random selection
    file, random_numbers, sat_num = args
    file_param = file.split("/")[1].split("_")
    h = float(file_param[0])
    beta = float(file_param[1])
    alpha0 = float(file_param[2].split(".npy")[0])
    param = [h, beta, alpha0]

    cover = cal_supply_all_cell(h, beta, alpha0)
    fulltime_cover = recover_fulltime([random_numbers, cover, n_length])
    
    return [param, random_numbers, fulltime_cover, sat_num]


In [18]:
sat_height=573
# 需求函数
base_cover_path="candidate_backbone_network_"+str(sat_height)+"/"

time_split=308
print(time_split)
demand_list=np.load("data/backbone_satellite_demand.npy",allow_pickle=True)
d=1
selected_params=np.load("data/TinyLEO_for_backbone_network.npy",allow_pickle=True)
print(len(selected_params))
files=[]
sat_nums=0
for s_tmp in selected_params:
    param_tmp,slots,satmaps,sat_num=s_tmp
    tmp=base_cover_path+str(int(param_tmp[0]))+"_"+str(param_tmp[1])+"_"+str(param_tmp[2])+".npy"
    files.append([tmp,slots,sat_num])
    sat_nums+=sat_num
print(files[0])
print(sat_nums)
all_cover=[]
n_length=len(demand_list)*time_split

with mp.Pool(processes=100) as pool:
    all_cover = list(tqdm(pool.imap(generate_baseset, files), total=len(files), mininterval=10,maxinterval=20))
    pool.close()  # Prevents any more tasks from being submitted to the pool
    pool.join()  # Wait for the worker processes to exit

308
3343
['candidate_backbone_network_573/573_0.9773843811168246_2.443460952792061.npy', [0], 1]
3343


100%|██████████| 3343/3343 [00:01<00:00, 2533.86it/s]


In [19]:
# === Flatten demand vector into time-expanded form ===
tmp_R = [demand for demand in demand_list]
ttmpR = tmp_R * int(time_split)
R = np.array(ttmpR).reshape(-1)
standard_R = sum(R)
del ttmpR, tmp_R  # Clean up

selected = []  # Selected coverage combinations
num_processes = 110  # Number of parallel processes

# === Dot product: used to evaluate coverage gain ===
def compute_dot(args, show=False):
    R, cid = args
    global all_cover
    fulltime_cover = all_cover[cid][2][0]
    dot = fulltime_cover @ R  # Compute inner product (coverage gain)
    return [dot[0], cid]

# === Update residual demand ===
def update_residual(R, cid):
    global all_cover
    # Subtract selected satellite's contribution, weighted by its satellite count
    R[all_cover[cid][2][0].indices] -= (all_cover[cid][2][0].data * all_cover[cid][3])
    R = np.maximum(R, 0)  # Ensure no negative demand
    return R

# === Find max dot product in a result chunk ===
def find_max_in_chunk(chunk):
    chunk = np.array(chunk)
    max_value = np.max(chunk[:, 0])      # Max dot product value
    local_index = np.argmax(chunk[:, 0]) # Index within chunk
    max_index = chunk[local_index][1]    # Original global index
    return [max_value, int(max_index)]


In [20]:
# === Initialize time-expanded demand vector ===
tmp_R = [demand for demand in demand_list]
ttmpR = tmp_R * int(time_split)
R_2 = np.array(ttmpR).reshape(-1).copy()
R_sum_2 = np.sum(R_2)
print("Initial residual:", R_sum_2)
del ttmpR, tmp_R  # Cleanup

process = []  # Store coverage progress

# === Sequentially subtract coverage from each selected base ===
for idx in range(len(all_cover)):
    R_2 = update_residual(R_2, idx)  # Update residual demand
    covered = R_sum_2 - np.sum(R_2)  # Coverage gained this step
    process.append([all_cover[idx][3], covered])
    R_sum_2 = np.sum(R_2)
    print("Residual after step", idx, ":", R_sum_2)

# === Summary of remaining demand ===


Initial residual: 153692.0
Residual after step 0 : 153599.0
Residual after step 1 : 153506.0
Residual after step 2 : 153413.0
Residual after step 3 : 153320.0
Residual after step 4 : 153227.0
Residual after step 5 : 153134.0
Residual after step 6 : 153041.0
Residual after step 7 : 152948.0
Residual after step 8 : 152855.0
Residual after step 9 : 152762.0
Residual after step 10 : 152669.0
Residual after step 11 : 152576.0
Residual after step 12 : 152483.0
Residual after step 13 : 152390.0
Residual after step 14 : 152297.0
Residual after step 15 : 152204.0
Residual after step 16 : 152111.0
Residual after step 17 : 152018.0
Residual after step 18 : 151925.0
Residual after step 19 : 151832.0
Residual after step 20 : 151739.0
Residual after step 21 : 151646.0
Residual after step 22 : 151553.0
Residual after step 23 : 151460.0
Residual after step 24 : 151367.0
Residual after step 25 : 151274.0
Residual after step 26 : 151181.0
Residual after step 27 : 151088.0
Residual after step 28 : 150995

In [23]:
print(f"Remaining nonzero entries = {np.count_nonzero(R_2)}, Max = {np.max(R_2):.2f}, Mean = {np.mean(R_2):.2f}")

Remaining nonzero entries = 0, Max = 0.00, Mean = 0.00


In [22]:
plt_data=[0]
r_sum=sum([demand for demand in demand_list])*time_split
for item in process:
    for _ in range(item[0]):
        v=item[1]/item[0]/r_sum*100+plt_data[-1]
        plt_data.append(v)
np.save("data/network_availability_backbone_demand.npy",plt_data)

In [24]:
time_shift=0
total_supply=[0]*len(demand_list)
for cover in tqdm(all_cover):
    cover_csr = cover[2][0]
    sat_num = cover[3]
    total_supply += cover_csr.toarray().flatten()[len(demand_list)*time_shift:len(demand_list)*(time_shift+1)] * sat_num  

100%|██████████| 3343/3343 [00:00<00:00, 29706.75it/s]


In [26]:
supply_demand_ratio=[]
for idx,demand in enumerate(demand_list):
    if demand!=0:
        supply_demand_ratio.append(total_supply[idx]/demand)
print(np.mean(supply_demand_ratio))
np.save("data/supply_demand_ratio_backbone.npy",supply_demand_ratio)

7.505528028121609


In [28]:
def FeasibilityCheck_one_slot(params):
    t,mega=params
    two_pi=2*np.pi
    total_supply=[]
    for idx,shell in enumerate(mega):
        T=T_list[idx]
        inc=inc_list[idx]
        for mid in range(shell["m"]):
            for nid in range(shell["n"]):
                r_set=[]
                a_sat=(two_pi/shell["m"]*mid-t/TE*two_pi)%two_pi
                b_sat=(two_pi/shell["n"]*nid+t/T*two_pi)%two_pi
                l_sat,p_sat=alpha_gamma_to_lambda_phi(a_sat,b_sat,inc)
                cid=create_new_grid_satellites([l_sat,p_sat])
                r_set=[[cid,1.0]]
                total_supply.append(r_set)
    # check the demand
    supply=[0]*len(demand_list)
    for r_set in total_supply:
        for item in r_set:
            supply[item[0]]+=item[1]
    return supply

In [31]:
backbone_data=np.load("data/MegaReduce_official_demand.npy",allow_pickle=True)
selected_data_backbone=backbone_data[-1][0]
T_list = np.array([satellite_period(shell["h"] * 1e3) for shell in selected_data_backbone])
inc_list = np.radians(np.array([shell["inc"] for shell in selected_data_backbone]))
total_supply_backbone=FeasibilityCheck_one_slot([0,selected_data_backbone])

supply_demand_ratio=[]
for idx,demand in enumerate(demand_list):
    if demand!=0:
        supply_demand_ratio.append(total_supply_backbone[idx]/demand)
print(np.mean(supply_demand_ratio))
np.save("data/supply_demand_ratio_backbone_megareduce.npy",supply_demand_ratio)

13.836079205998992
